<a href="https://colab.research.google.com/github/aubreymoore/crb-yona-tiff/blob/main/slicing_tif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import os
from osgeo import gdal

In [34]:
def get_data():
  if not os.path.exists('crb-yona-tiff'):
    print('cloning GitHub repo')
    os.system('git clone https://github.com/aubreymoore/crb-yona-tiff.git')
  if not os.path.exists('yona=tif'):
    print('assembling geoTIFF')
    os.system('cat crb-yona-tiff/chunks/* > yona.tif')
  print('loading data')
  rds = gdal.Open('yona.tif')
  return rds

In [29]:
def get_metadata(rds):
  width = rds.RasterXSize
  height = rds.RasterYSize
  geotransform = rds.GetGeoTransform()
  return width, height, geotransform

In [30]:
def get_geo_transform(rds):
  ulcorner_x, pixel_width, row_rotation, ulcorner_y, col_rotation, pixel_height = rds.GetGeoTransform()
  print(f'gt[0] x-coordinate of the upper-left corner of the upper-left pixel:             {gt[0]} meters')
  print(f'gt[1] w-e pixel resolution / pixel width:                                        {gt[1]} meters')
  print(f'gt[2] row rotation (typically zero):                                             {gt[2]}')
  print(f'gt[3] y-coordinate of the upper-left corner of the upper-left pixel:             {gt[3]} meters')
  print(f'gt[4] column rotation (typically zero):                                          {gt[4]}')
  print(f'gt[5] n-s pixel resolution / pixel height (negative value for a north-up image): {gt[5]} meters')
  return ulcorner_x, pixel_width, row_rotation, ulcorner_y, col_rotation, pixel_height

get_geo_transform(rds)

gt[0] x-coordinate of the upper-left corner of the upper-left pixel:             259821.30138992154 meters
gt[1] w-e pixel resolution / pixel width:                                        0.014080873475957724 meters
gt[2] row rotation (typically zero):                                             0.0
gt[3] y-coordinate of the upper-left corner of the upper-left pixel:             1484150.4414387643 meters
gt[4] column rotation (typically zero):                                          0.0
gt[5] n-s pixel resolution / pixel height (negative value for a north-up image): -0.014080873475957724 meters


(259821.30138992154,
 0.014080873475957724,
 0.0,
 1484150.4414387643,
 0.0,
 -0.014080873475957724)

In [31]:
def pix2geo(pix_x, pix_y):
  geo_x = ulcorner_x + pix_x * pixel_width
  geo_y = ulcorner_y + pix_y * pixel_height
  return geo_x, geo_y

def geo2pix(geo_x, geo_y):
  pix_x = int((geo_x - ulcorner_x) / pixel_width)
  pix_y = int((geo_y - ulcorner_y) / pixel_height)
  return pix_x, pix_y

In [32]:
# MAIN

rds = get_data()
width, height, geotransform = get_metadata(rds)

# unpack the geotransform
ulcorner_x, pixel_width, row_rotation, ulcorner_y, col_rotation, pixel_height = geotransform




cloning GitHub repo
assembling geoTIFF
loading data


In [42]:
print(height, width)
xc = width // 2
yc = height // 2
x1 = xc - 499
x2 = xc + 500
y1 = yc - 499
y2 = yc + 500



23585 32731


In [44]:
# window = (offset_x, offset_y, size_x, size_y)
# gdal.Translate('output_crop_raster.tif', 'input_raster.tif', srcWin = window)

window = (xc-500, yc-500, 1000, 1000)
gdal.Translate('center.tif', 'yona.tif', srcWin = window)


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f34c7099080> >

In [45]:
!ls

center.tif  crb-yona-tiff  sample_data	yona.tif
